# `pygvec` example for using B-spline profiles

This notebook demonstrates how `pygvec` interfaces with the B-spline radial plasma profiles available in `gvec`.

In [ ]:
import os

from scipy.interpolate import make_splrep
import numpy as np
import matplotlib.pyplot as plt

# OMP number of threads for gvec run needs to be before import of gvec
os.environ["OMP_NUM_THREADS"] = "2"
# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing

First, we want to interpolate some pressure profile shape and hand it over to `gvec`. To this end we define a simple profile parameterization, a two-power profile:

In [ ]:
def two_power(s, a=3, b=5):
    return (1 - s**a) ** b


# first derivative with respect to rho=sqrt(s)
def two_power_dr(s, a=3, b=5):
    return b * (1 - s**a) ** (b - 1) * (-a * s ** (a - 1)) * 2 * np.sqrt(s)


# second derivative with respect to rho=sqrt(s)
def two_power_drr(s, a=3, b=5):
    return (
        2 * a * b * s ** (a - 1) * (1 - s**a) ** (b - 2) * (2 * a * (b * s**a - 1) - s**a + 1)
    )


x_tp = np.linspace(0, 1, 51)
y_tp = two_power(x_tp)

x_plot = np.linspace(0, 1, 100)
y_plot = two_power(x_plot)

fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

Next, we interpolate the profile with B-splines using `scipy`. 

In [ ]:
bspl = make_splrep(x_tp, y_tp, k=5)
y_bspl = bspl(x_plot)

fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.plot(x_plot, y_bspl, "r--", label="scipy B-Spline fit")
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

Now we want to adapt an existing parameter file with some changed values and our B-spline fit of the two-power pressure profile:

In [ ]:
params = {}
iota_coefs = [0.523, 0.625]  # a0*s + a1

# This profile uses the default polynomial representation
params["iota_coefs"] = iota_coefs  # c_0 + c_1*s +c_2*s^2+ ...
params["pres_scale"] = 1500
params

To translate a `scipy` B-spline into input parameters as used by `gvec` we can call `gvec.util.bspl2gvec`.

In [ ]:
params = gvec.util.bspl2gvec(
    "pres", bspl, params=params
)  # handing over the params dict extends it
params

Alternatively we could just hand over the knots and coefficients to `gvec.util.bspl2gvec`, if, for example, we used an other interpolation routine.

In [ ]:
params = gvec.util.bspl2gvec("pres", knots=bspl.t, coefs=bspl.c, params=params)
params

With the new parameters we can thus evaluate the profile and its derivatives with respect to $\rho=\sqrt{s}$ or run `gvec`.

In [ ]:
gvec.util.adapt_parameter_file("parameter.ini", "parameter_bspl.ini", **params)

In [ ]:
with gvec.State("parameter_bspl.ini") as state:
    rho = np.linspace(0, 1, 100)  # radial visualization points
    p = state.evaluate_profile("p", rho=rho)
    dp_dr = state.evaluate_profile("p", rho=rho, deriv=1)
    dp_drr = state.evaluate_profile("p", rho=rho, deriv=2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.plot(
    rho**2, p / p[0], "r--", label="gvec B-spline"
)  # note that the pressure profile is scaled with pres_scale
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

In [ ]:
fig_dr, ax_dr = plt.subplots()
ax_dr.plot(x_plot, two_power_dr(x_plot), label="two power profile")
ax_dr.plot(rho**2, dp_dr / p[0], "r--", label="gvec B-spline")
ax_dr.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax_dr.set_ylabel(r"$P'/P_0$")
plt.legend()

fig_drr, ax_drr = plt.subplots()
ax_drr.plot(x_plot, two_power_drr(x_plot), label="two power profile")
ax_drr.plot(rho**2, dp_drr / p[0], "r--", label="gvec B-spline")
ax_drr.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax_drr.set_ylabel(r"$P''/P_0$")
plt.legend()

`gvec` also provides the option to directly interpolate the $\iota$ and pressure profile via cubic B-splines. In such a case we need to provide `iota_vals`/`pres_vals` and `iota_rho2`/`pres_rho2` and change `iota_type`/`pres_type` to `interpolation`. Per default the interpolation uses *not-a-knot* boundary conditions.

In [ ]:
# delete the old entries
del params["pres_knots"]
del params["pres_coefs"]

With our two power profile example we hand over `x_tp` for $\rho^2=$`rho2` and `y_tp` as `vals`:

In [ ]:
params["pres_vals"] = y_tp
params["pres_rho2"] = x_tp
params["pres_type"] = "interpolation"
gvec.util.adapt_parameter_file("parameter.ini", "parameter_interpolation.ini", **params)

Let's see if the interpolation is as expected:

In [ ]:
rho = np.linspace(0, 1, 100)  # radial visualization points

with gvec.State("parameter_interpolation.ini") as state:
    p = state.evaluate_profile("p", rho=rho)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.plot(
    rho**2, p / p[0], "r--", label="gvec interpolation"
)  # note that the pressure profile is scaled with pres_scale
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

Alternatively to the *not-a-knot* boundary condition (BC) one can also specify the first or second derivative at the axis/edge via the `_BC_type_axis` or `_BC_type_edge` and the array `_BC_vals`. For example in the two power profile example we could set the $1^{\text{st}}$ derivative at the edge to zero by specifying `pres_BC_type_edge=1st_deriv`. The default BC value is zero, but for demonstration purposes we will still specify it explicitly via `pres_BC_vals=(/0.0,0.0/)`. Note that since `pres_BC_type_axis=not_a_knot`, the first entry of `pres_BC_vals` is ignored.

To see some differences between the BC, let us also reduce the number of available interpolation points:

In [ ]:
x_tp = np.linspace(0, 1, 4)
y_tp = two_power(x_tp)

bspl = make_splrep(x_tp, y_tp)

In [ ]:
params["pres_vals"] = y_tp
params["pres_rho2"] = x_tp
params["pres_BC_type_edge"] = "1st_deriv"
params["pres_BC_vals"] = [0.0, 0.0]
gvec.util.adapt_parameter_file("parameter.ini", "parameter_interpolation.ini", **params)

with gvec.State("parameter_interpolation.ini") as state:
    p = state.evaluate_profile("p", rho=rho)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.plot(rho**2, bspl(rho**2), "-.", label="scipy not-a-knot")
ax.plot(
    rho**2, p / p[0], "r--", label=r"gvec $\text{BC}_{edge}: P'(\rho^2 = 1)=0$"
)  # note that the pressure profile is scaled with pres_scale
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

In our example, we can improve the interpolation even further by also specifying the $1^{\text{st}}$ derivative at the axis.

In [ ]:
params["pres_BC_type_axis"] = "1st_deriv"
gvec.util.adapt_parameter_file("parameter.ini", "parameter_interpolation.ini", **params)

with gvec.State("parameter_interpolation.ini") as state:
    p = state.evaluate_profile("p", rho=rho)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, label="two power profile")
ax.plot(x_tp, y_tp, ".", label="interpolation points")
ax.plot(rho**2, bspl(rho**2), "-.", label="scipy not-a-knot")
ax.plot(
    rho**2, p / p[0], "r--", label=r"gvec $\text{BC}_{edge} = \text{BC}_{axis}: P'=0$"
)  # note that the pressure profile is scaled with pres_scale
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()